In [ ]:
import sqlite3
import pandas as pd
import numpy as np

In [ ]:
def blend_table(conn_old, conn_new, table_name,
                key_cols=('Country','FIPS_CODE'),
                meta_cols=('Earliest','MostRecent')):
    """
    Blend table_name from conn_old into conn_new:
      - Detect all numeric 'year' columns in both tables
      - Build full year-range (min → max) and include any non-year data cols
      - Reindex both tables so they share exactly those columns (gap-years become NaN)
      - Wherever new is NaN, backfill from old
      - Recompute Earliest & MostRecent from the full year set
      - Drop & recreate table in conn_new
    """
    # ─── 1. Load old & new tables ──────────────────────────────────────────────
    df_old = pd.read_sql_query(f"SELECT * FROM [{table_name}]", conn_old)
    df_new = pd.read_sql_query(f"SELECT * FROM [{table_name}]", conn_new)

    # ─── 2. Drop any existing meta columns in df_new so they get rebuilt later ─
    df_new = df_new.drop(columns=[c for c in meta_cols if c in df_new], errors='ignore')

    # ─── 3. Identify candidate data columns (everything except keys + meta) ───
    cols_old = set(df_old.columns) - set(key_cols) - set(meta_cols)
    cols_new = set(df_new.columns) - set(key_cols) - set(meta_cols)

    # ─── 4. Split year-columns vs. other data-columns ─────────────────────────
    #    year-cols are those whose names are all digits
    year_cols_old = [c for c in cols_old if c.isdigit()]
    year_cols_new = [c for c in cols_new if c.isdigit()]
    all_year_nums = [int(y) for y in year_cols_old + year_cols_new]

    if all_year_nums:
        year_min, year_max = min(all_year_nums), max(all_year_nums)
        full_year_cols = [str(y) for y in range(year_min, year_max+1)]
    else:
        full_year_cols = []

    other_data_cols = sorted((cols_old | cols_new) - set(full_year_cols))

    all_data_cols = other_data_cols + full_year_cols

    # ─── 5. Reindex both DFs to have exactly key_cols + all_data_cols ──────────
    df_old = df_old.reindex(columns=list(key_cols) + all_data_cols)
    df_new = df_new.reindex(columns=list(key_cols) + all_data_cols)

    # ─── 6. Align on the key index & combine (new overrides, then old fills) ───
    df_old_i = df_old.set_index(list(key_cols))
    df_new_i = df_new.set_index(list(key_cols))
    df_blend_i = df_new_i.combine_first(df_old_i)

    # ─── 7. Back to a flat DataFrame ─────────────────────────────────────────
    df_blend = df_blend_i.reset_index()

    # ─── 8. Ensure numeric dtype on the year columns ──────────────────────────
    for c in full_year_cols:
        df_blend[c] = pd.to_numeric(df_blend[c], errors='coerce')

    # ─── 9. Recompute Earliest & MostRecent from only the year sequence ───────
    if full_year_cols:
        df_blend['Earliest'] = (
            df_blend[full_year_cols]
            .apply(lambda row: row.dropna().iloc[0] if row.dropna().size else np.nan, axis=1)
        )
        df_blend['MostRecent'] = (
            df_blend[full_year_cols]
            .apply(lambda row: row.dropna().iloc[-1] if row.dropna().size else np.nan, axis=1)
        )
    else:
        df_blend['Earliest'] = np.nan
        df_blend['MostRecent'] = np.nan

    # ─── 10. Write back: drop + recreate + append ──────────────────────────────
    cur = conn_new.cursor()
    cur.execute(f"DROP TABLE IF EXISTS [{table_name}];")

    # Build CREATE TABLE DDL
    ddl_cols = (
        ", ".join(f"[{col}] DOUBLE" for col in all_data_cols)
        + ", [Earliest] DOUBLE, [MostRecent] DOUBLE"
    )
    create_sql = (
        f"CREATE TABLE [{table_name}] "
        f"(Country VARCHAR(255), FIPS_CODE VARCHAR(255), {ddl_cols});"
    )
    cur.execute(create_sql)

    # Bulk-insert
    df_blend.to_sql(table_name, conn_new, if_exists='append', index=False)
    conn_new.commit()


def needs_blending(conn_old, conn_new, table_name,
                   key_cols=('Country','FIPS_CODE')):
    """
    Return True if:
      • there's any year-col in old NOT in new, or
      • for any shared year-col, new has NaN but old has a value.
    """
    # Load both tables
    df_old = pd.read_sql_query(f"SELECT * FROM [{table_name}];", conn_old)
    df_new = pd.read_sql_query(f"SELECT * FROM [{table_name}];", conn_new)

    # Identify year-columns (names all digits)
    years_old = {c for c in df_old.columns if c.isdigit()}
    years_new = {c for c in df_new.columns if c.isdigit()}

    # 1) Old-only year columns → need to blend
    if years_old - years_new:
        return True

    # 2) Check overlapping years for fillable gaps
    common_years = sorted(years_old & years_new)
    if not common_years:
        return False

    # Align on key and build mask in one go
    df_old_i = df_old.set_index(list(key_cols))
    df_new_i = df_new.set_index(list(key_cols))
    mask = df_new_i[common_years].isna() & df_old_i[common_years].notna()

    return mask.values.any()


def blend_databases(path_old, path_new, tables=None):
    conn_old = sqlite3.connect(path_old)
    conn_new = sqlite3.connect(path_new)

    if tables is None:
        dd = pd.read_sql_query("SELECT [Table] FROM DataDict;", conn_new)
        tables = dd['Table'].tolist()

    for tbl in tables:
        if not needs_blending(conn_old, conn_new, tbl):
            print(f"↩️  Skipping {tbl}: no missing columns or fillable gaps.")
            continue

        try:
            blend_table(conn_old, conn_new, tbl)  # your full blend_table()
            print(f"✔ Blended {tbl}")
        except Exception as e:
            print(f"✖ Failed {tbl}: {e}")

    conn_old.close()
    conn_new.close()

In [ ]:
old_db = r"YOUR_PATH/IFsHistSeries.db" # full path to the old IFsHistSeries.db
new_db = r"YOUR_PATH/IFsDataImport.db" # path to the IFsDataImport.db
blend_databases(old_db, new_db)